In [1]:
!pip install openpyxl

In [2]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import os
import glob
import boto3
import s3fs
import tarfile
import joblib
import tempfile
# **IMPORTANT**: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED'] = str(2)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
from time import gmtime, strftime
import sagemaker
from sagemaker.sklearn import SKLearn
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sklearn.preprocessing import MinMaxScaler
from pickle import dump, load
import warnings
from config_module import Config
config = Config()

import random

# Set seed for reproducibility
np.random.seed(42)
pd.options.mode.chained_assignment = None

# Import modules
import itertools
import math
import pickle
from datetime import datetime, timezone, timedelta

from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import (mean_absolute_error, mean_absolute_percentage_error,
                             mean_squared_error)
from sklearn.model_selection import (GridSearchCV, TimeSeriesSplit,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
random.seed(12)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
start_time = datetime.now()

In [4]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    
    return data

In [5]:
def get_latest_file(folder_path, file_prefix):
    # Construct the search pattern for the CSV files with the given prefix
    search_pattern = os.path.join(folder_path, f"{file_prefix}*")
    
    
    # Get the list of matching files
    csv_files = glob.glob(search_pattern)
    
    
    if not csv_files:
        print("No CSV files found.")
        return None
    
    # Extract the datetime part and convert it to a datetime object
    csv_files_with_dates = []
    for file in csv_files:
        try:
            # Extract the datetime stamp from the file name
            datetime_stamp = file[len(folder_path) + len(file_prefix) + 1:-5]
            file_datetime = datetime.strptime(datetime_stamp, "%y%m%d_%H%M%S")
            csv_files_with_dates.append((file, file_datetime))
        except ValueError:
            print(f"Could not parse date from file name: {file}")
    
    if not csv_files_with_dates:
        print("No valid CSV files found.")
        return None
    
    # Sort the files by datetime in descending order and get the latest one
    latest_file = max(csv_files_with_dates, key=lambda x: x[1])[0]
    
    return latest_file

In [6]:
def get_latest_s3_file(bucket_name, path_prefix, file_prefix):
    s3 = boto3.client('s3')
    
    # List all objects under the given path_prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=path_prefix)
    
    if 'Contents' not in response:
        print("No files found.")
        return None
    
    # Filter objects that start with the specific file path_prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{path_prefix}{file_prefix}")
    ]
    
    if not filtered_files:
        print("No files found with the specific path_prefix.")
        return None
    
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")

    if not files_with_dates:
        print("No valid files with dates found.")
        return None

    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    
    return latest_file_key

In [7]:
def create_lagged_features(data, lag, pred_win):
    """
    This function creates lagged features for the given time series data.
    
    Parameters:
    data (pd.Series): The input time series data.
    lag (int): The number of lagged periods to create.
    pred_win (int): The prediction window size.
    
    Returns:
    pd.DataFrame: A DataFrame containing the lagged features.
    """
    
    # Initialize an empty DataFrame with the same index as the input data
    lagged_data = pd.DataFrame(index=data.index)
    
    # Create lagged features by shifting the data by (lag + pred_win - 1) periods
    for i in range(1, lag + 1):
        lagged_data[f'lag_{i}'] = data.shift(i + (pred_win - 1))
    
    return lagged_data

# Function to forecast future values using a model
def forecast(df, model, n_periods=3):
    """
    This function forecasts future values using the provided model.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame containing the time series data.
    model (object): The trained model to be used for forecasting.
    n_periods (int): The number of periods to forecast.
    
    Returns:
    list: A list of forecasted values.
    """
    # Get the last 'n_periods' values from the 'Total_qty_per_month' column
    last_values = df['Total_qty_per_month'][-3:].values
    
    # Initialize an empty list to store the forecasted values
    forecasts = []
    
    # Generate forecasts for the specified number of periods
    for _ in range(n_periods):
        # Extract the features for the model (last 'lag' values)
        # features = last_values[-lag:]
        features = last_values[-3:]
        
        # Predict the next value using the model
        pred = model.predict(features.reshape(1, -1))[0]
        
        # Append the predicted value to the forecasts list
        forecasts.append(pred)
        
        # Update the last values by appending the predicted value and removing the oldest value
        last_values = np.append(last_values, pred)[1:]
    
    return forecasts

In [8]:
bucket = config.get_value('bucket')
prefix_output_file = config.get_value('prefix_cleaned_training_file_path')
file_name_prefix = config.get_value('cleaned_training_filename_prefix')
file_path = get_latest_s3_file(bucket,prefix_output_file,file_name_prefix)

final_data = pd.read_excel('s3://'+f'{bucket}/{file_path}')
parts_list = final_data['part_number'].unique().tolist()

In [9]:
file_path

'persist-bo/persist-bo-demand-forecast/cleaned-training-data/clean_data_imputed_2024-11-06.xlsx'

In [10]:
len(parts_list)

6525

In [11]:
training_end_date = (pd.to_datetime(final_data['period']).max() + relativedelta(months=-3)).strftime('%Y-%m-%d')

In [12]:
def get_consecutive_months(start_date, n_months=3):
    """
    Generate a list of consecutive month start dates.

    Args:
    - start_date (str): Starting date in 'YYYY-MM-DD' format.
    - n_months (int): Number of consecutive months to generate.

    Returns:
    - list: List of consecutive month start dates in 'YYYY-MM-DD' format.
    """
    start_date = pd.to_datetime(start_date)
    month_list = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m-%d') for i in range(1, n_months + 1)]
    return month_list

In [13]:
print(datetime.now())

2024-11-06 14:02:07.386430


In [14]:
# Ridge Lag 1
#final_ridge_lag1
def ridge_lag1_func(clean_data, s3_bucketname, pickle_path_ridge_lag1, last_training_date):
    df1=clean_data[:]
    df1=df1[['part_number','period','Total_qty_per_month']]
    df1=df1.sort_values('period')
    df1['period'] = pd.to_datetime(df1['period'])
    # df1 = df1[df1['period']<=last_test_date]
    s3_client = boto3.client('s3')
    fs = s3fs.S3FileSystem()
    #create a model folder for saving pickle with a changed name
    model_folder = pickle_path_ridge_lag1[:]
    last_test_date = (pd.to_datetime(last_training_date) + relativedelta(months=3)).strftime('%Y-%m-%d')
    future_dates = get_consecutive_months(last_test_date)
    final_metric_lag1_ridge=pd.DataFrame()
    for i in df1['part_number'].unique():
        try:
            print(i)
            data= df1[df1['part_number']==i]            
            data.drop('part_number',axis=1,inplace=True)
            data['month'] = data['period'].dt.month
            
            # Step 1: Create and combine lagged features with original data
            lag = 3
            pred_win=1 #prediction window needs to be updated      
            lagged_data = create_lagged_features(data['Total_qty_per_month'], lag ,pred_win)
            data_with_lags = pd.concat([data, lagged_data], axis=1)
            data_with_lags.fillna(data_with_lags.median(), inplace=True)
            
            # Step 2: Split the Data to create training and testing datasets
            train_size = len(data_with_lags)-3
            train_data = data_with_lags.iloc[:train_size]
            test_data = data_with_lags.iloc[train_size:]
            X_train_scaled = train_data.drop(['period', 'month','Total_qty_per_month'], axis=1)
            X_test_scaled= X_train_scaled[-3:]
            y_train = train_data['Total_qty_per_month']
            #y_test = test_data[:]
            
            # Step 3: Fit the Ridge Regression Model
            ridge = Ridge(alpha=1,random_state=42)  # Adjust alpha as needed
            ridge.fit(X_train_scaled, y_train)
            
            ################
            output_file = os.path.join(f"s3://{s3_bucketname}/{pickle_path_ridge_lag1}/", f"ridge-t-{i}.joblib")
            output_file_temp = f"/tmp/ridge-t-{i}.joblib"
            with fs.open(output_file, 'wb') as f:
                joblib.dump(ridge, f)
            with open(output_file_temp, 'wb') as model_file:
                joblib.dump(ridge, model_file)
            tar_model_path = output_file_temp
            with tarfile.open(tar_model_path, "w:gz") as tar:
                tar.add(output_file_temp, arcname=os.path.basename(output_file_temp))
            # Upload the tar.gz archive to S3
            s3_model_path = f"{pickle_path_ridge_lag1}/ridge-t-{i}.tar.gz"
            s3_client.upload_file(tar_model_path, s3_bucketname, s3_model_path)
            ################
            
            # Step 5: Evaluate the Model
            y_pred = ridge.predict(X_test_scaled)
            X_pred = ridge.predict(X_train_scaled)
            
            # Step 6: Forecast the next 3 periods and create a consolidated list of predicted values
            n_periods = 3
            forecasts = forecast(data_with_lags, ridge, n_periods)        
            X_prediction=X_pred.tolist()
            test_pred= y_pred.tolist()
            Y_prediction=forecasts
            X_Y_pred=X_prediction+test_pred+Y_prediction
            
            # Step 7: Create a final dataframe with part-wise Actual and fitted values
            metric=pd.DataFrame()
            dates =pd.DataFrame(data['period'])
            new_date = pd.DataFrame({'period': pd.to_datetime(future_dates)})
            dates = pd.concat([dates, new_date], ignore_index=True)
            metric['period']=dates['period']
            qty = pd.DataFrame(data['Total_qty_per_month'])
            new_qty = pd.DataFrame({'Total_qty_per_month': [0,0,0]})
            qty = pd.concat([qty, new_qty], ignore_index=True)
            metric['Actual_values']=qty['Total_qty_per_month']
            metric['Predicted_values_lag1']= pd.DataFrame(X_Y_pred)
            metric['part_number']=i
            final_metric_lag1_ridge=pd.concat([final_metric_lag1_ridge,metric])
        except Exception as e:
            print(f"{i} part number has error: {e}")
    return final_metric_lag1_ridge

In [15]:
print(datetime.now())

2024-11-06 14:02:07.405412


In [16]:
training_date = pd.to_datetime(config.get_value('model_training_date'))
file_date = training_date.strftime("%Y-%m-%d")

In [ ]:
# final_data_subset = final_data[final_data['part_number'].isin(parts_list[:100])]  # line to be commented post testing
final_data_subset = final_data[final_data['part_number'].isin(parts_list[:])]
prefix_ridge_pickle = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{'ridge'}"
prefix_output = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{config.get_value('model_output_folder_name')}"

# print(prefix_ridge_pickle)

ridge_data=ridge_lag1_func(final_data_subset, bucket, prefix_ridge_pickle, training_end_date)

file_name_train_ridge_prefix = f'{config.get_value("file_name_train_ridge_prefix")}{file_date}.xlsx'
file_path_train_ridge = f's3://{bucket}/{prefix_output}/{file_name_train_ridge_prefix}'
ridge_data.to_excel(file_path_train_ridge, index=False)

PZT0038023
473110K010
47311KK020
90364T0024
47313KK030
47314KK020
473160K030
90364T0028
464300K170
47316KK020
473170K010
47322KK010
90366T0008
473290K011
473290K021
473410K010
130410Y04003
47265200501
90311T0080
464300K150
464300K120
462430K010B0
463040K030
743200K020B0
4640252030
464100K011
464100K120
4641035420A1
464300K100
4642002310
4642006070
464200A020
464200D260
90364T0027
464200D270
464200K011
464200K030
4642004050
473410K080
473430K080
473210D360
130410Y04001
472010K040
4712112020A1
47351KK010
049450K321
458970K010
455360K010
452030K040
044660D040
452030K010
451870K050C0
4518706050B0
451860K120C0
11201BZ100
451860D050C0
4518606150C0
4518606090B0
123640Y010
13520BZ030
464200K041
472010K030
472010K010
462010K230C0
47201BZ090
472010K380
130410C06006
464300K220
4643035330A1
465400A010
13011BZ161
465560K012
465590K012
465900K010
472010K590
123720Y033
471210D050
471480K011
472000K460
15100BZ060
472010K020
451840K130C0
746110K030E0
464200K091
131010E01001
131010E01002
130410E02106
13

In [ ]:
# ridge_data['part_number'].nunique()

In [ ]:
print(datetime.now())

In [ ]:
end_time = datetime.now()
print("Total time taken for Ridge training::", (end_time-start_time).seconds/60, " minutes")